In [18]:
#IMPORTAR BIBLIOTECAS UTILIZADAS

import numpy as np
import pandas as pd
import random
from collections import Counter
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
import time

In [19]:
"""
    Classe para processar o dataset
    Obs: o dataset tem que ser para problemas de classificacao e nele tem que conter uma coluna Class
"""
import os
import sys
sys.path.insert(0, os.path.abspath('../Util'))
from dados import ProcessarDados

In [32]:
#IMPORTANDO DATASET

dataset = ProcessarDados("../dataset/bin_norm_10_features_m73_n4.csv", sep = ',')

In [33]:
# função que treina e testa o modelo armazenando as métricas
# retorna um dicionário cotendo os valores das métricas de cada rodada

def treinamento_teste(epocas = 10, k_folds = 5, exibir_matriz_confusao=False, exibir_metricas=False):
    
    #array para armazenar as das métricas de cada rodada
    resultados_accuracy = []
    resultados_precision = []
    resultados_recall = []
    resultados_specificity = [] # taxa de verdadeiros negativos ou especificidade
    resultados_f2 = []
    resultados_parametros = []
    resultados_time_train = []
    resultados_time_test = []
    resultados_time_total = []
    
    #dicionário das métricas
    resultados_gerais = {}

    for i in range(epocas):
        #SEPARAR DADOS DE TREINO E TESTE MANTENDO A MESMA PROPORÇÃO  (80% E 20%)
        seed = i
        X_treino, X_teste, y_treino, y_teste = dataset.holdout2(0.2, seed)
        
        #ALTERANDO DIMENSÃO DE Y PARA D=1

        #y_treino = y_treino.reshape(480,)
        #y_teste = y_teste.reshape(120,)
        
        # realizando o grid search para encontrar o melhor alpha, 
        # considerando a acurácia (taxa de acerto)
        # aqui o método GridSearchCV é configurado para subdividir os dados de treino em k_folds
        rl = LogisticRegression(random_state = seed)
        grid_rl = GridSearchCV(rl, param_grid, cv=k_folds, scoring='accuracy', verbose=0)
        
        start_time_train = time.clock()
        grid_rl.fit(X_treino, y_treino)

        # Treinando do modelo com os melhores parametros encontrados
        alpha_best = grid_rl.best_estimator_.tol

        rl = LogisticRegression(random_state = seed, tol = alpha_best)
        rl.fit(X_treino, y_treino)
        
        time_train = time.clock() - start_time_train

        #testando o modelo
        start_time_test = time.clock()
        y_pred = rl.predict(X_teste)
        time_test = time.clock() - start_time_test
        
        cm  = confusion_matrix(y_teste, y_pred)
        if exibir_matriz_confusao:
            print(cm)


        # calculado as metricas
        accuracy = metrics.accuracy_score(y_teste, y_pred)
        precision = metrics.precision_score(y_teste, y_pred)
        recall = metrics.recall_score(y_teste, y_pred)
        # f2-score
        # Fbeta = ((1 + beta^2) * Precision * Recall) / (beta^2 * Precision + Recall)
        beta = 0.5
        f2_score = (1 + beta**2) * (precision * recall) / (beta**2 * precision + recall)
        tn, fp, fn, tp = cm.ravel()
        specificity = tn / (tn+fp)
        

        # armazenando as métricas
        resultados_accuracy.append(accuracy)
        resultados_precision.append(precision)
        resultados_recall.append(recall)
        resultados_specificity.append(specificity)
        resultados_f2.append(f2_score)

        best_parametros = {'alpha' : alpha_best};
        resultados_parametros.append(best_parametros)
        
        resultados_time_train.append(time_train)
        resultados_time_test.append(time_test) 
        resultados_time_total.append(time_train+time_test)
        
        if exibir_metricas:
            print("Rodada: #",i)
            print(best_parametros)
            print("Accuracy:",accuracy)
            print("Precision:",precision)
            print("Recall:",recall)
            print("Specificity:",specificity)
            print("f2-Score:",f2_score)
            print("Time Train (s):",time_train)
            print("Time Test (s):",time_test)
            print("Time Total (s):",time_train+time_test)
            print("\n")

            
    resultados_gerais['accuracy'] = resultados_accuracy
    resultados_gerais['precision'] = resultados_precision
    resultados_gerais['recall'] = resultados_recall
    resultados_gerais['specificity'] = resultados_specificity
    resultados_gerais['f2'] = resultados_f2
    resultados_gerais['params'] = resultados_parametros
    resultados_gerais['time_train'] = resultados_time_train
    resultados_gerais['time_test'] = resultados_time_test
    resultados_gerais['time_total'] = resultados_time_total
    
    return resultados_gerais

In [34]:
import csv
def writeResultsCsv(dict_results, name_file):
    keys = list(dict_results.keys())
    keys.insert(0,'Rodada')
    with open(name_file, 'w') as f:  # You will need 'wb' mode in Python 2.x
        w = csv.writer(f)
        #w.writeheader()
        row = []
        w.writerow(keys)
        for i in range(epocas):
            row = []
            acc = dict_results.get('accuracy')[i]
            prec = dict_results.get('precision')[i]
            sen = dict_results.get('recall')[i]
            spe = dict_results.get('specificity')[i]
            fscore = dict_results.get('f2')[i]
            param = dict_results.get('params')[i]
            time_train = dict_results.get('time_train')[i]
            time_test = dict_results.get('time_test')[i]
            time_total = dict_results.get('time_total')[i]

            row.append(i+1)
            row.append(acc)
            row.append(prec)
            row.append(sen)
            row.append(spe)
            row.append(fscore)
            row.append(param)
            row.append(time_train)
            row.append(time_test)
            row.append(time_total)

            w.writerow(row)
    

In [35]:
def tabelaMetricas(nome_modelo, dict_metricas, rodadas=False, salvarResultados=True):
        
    print ("============================================== "+nome_modelo+" =================================================")
    print ("=================================== TABELA DE MÉTRICAS DO MODELO ===================================")
    
    if(rodadas==False):
        print ("\t Accuracy \t|\t Precision \t|\t Recall \t|\tSpecificity \t|\t fb-Score")
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['accuracy'], axis=0), np.std(dict_metricas['accuracy'], axis=0)),end='       ')
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['precision'], axis=0), np.std(dict_metricas['precision'], axis=0)),end='    ')
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['recall'], axis=0), np.std(dict_metricas['recall'], axis=0)),end='       ')
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['specificity'], axis=0), np.std(dict_metricas['specificity'], axis=0)),end='       ')
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['f2'], axis=0), np.std(dict_metricas['f2'], axis=0)))
        print ("====================================================================================================")
        
    if(salvarResultados):
        # save to npy file
        np.save('../resultados/resultados_'+nome_modelo+'.npy', dict_metricas)
        writeResultsCsv(dict_metricas, 'resultados_rl.csv')

In [36]:
#DEFININDO OS PARÂMETROS


alpha_range = np.logspace(-3,0,25)
param_grid = {'tol': alpha_range}

epocas = 50
k_folds = 5
exibir_matriz_confusao = False
exibir_metricas = True

In [37]:
# TREINANDO O MODELO E OBTENDO AS MÉTRICAS

dict_metricas = treinamento_teste(epocas, k_folds, exibir_matriz_confusao, exibir_metricas)
tabelaMetricas('RL',dict_metricas)

Rodada: # 0
{'alpha': 0.7498942093324559}
Accuracy: 0.9285714285714286
Precision: 0.9591836734693877
Recall: 0.94
Specificity: 0.9
f2-Score: 0.9552845528455283
Time Train (s): 1.0014976000002207
Time Test (s): 0.00017280000065511558
Time Total (s): 1.0016704000008758


Rodada: # 1
{'alpha': 1.0}
Accuracy: 0.8857142857142857
Precision: 0.9375
Recall: 0.9
Specificity: 0.85
f2-Score: 0.9297520661157026
Time Train (s): 0.8933818000004976
Time Test (s): 0.000119099999210448
Time Total (s): 0.893500899999708


Rodada: # 2
{'alpha': 0.1}
Accuracy: 0.9142857142857143
Precision: 0.9230769230769231
Recall: 0.96
Specificity: 0.8
f2-Score: 0.9302325581395349
Time Train (s): 0.936612100000275
Time Test (s): 0.00012850000166508835
Time Total (s): 0.93674060000194


Rodada: # 3
{'alpha': 0.31622776601683794}
Accuracy: 0.9142857142857143
Precision: 0.94
Recall: 0.94
Specificity: 0.85
f2-Score: 0.94
Time Train (s): 0.9801836000006006
Time Test (s): 0.00012789999891538173
Time Total (s): 0.9803114999995